<a href="https://colab.research.google.com/github/SamarAlamri/Ratq_Model/blob/main/5modelsForErythema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip -q install -U tensorflow==2.15.1 pandas numpy scikit-learn pillow tqdm matplotlib seaborn


ERROR: Could not find a version that satisfies the requirement tensorflow==2.15.1 (from versions: 2.16.0rc0, 2.16.1, 2.16.2, 2.17.0rc0, 2.17.0rc1, 2.17.0, 2.17.1, 2.18.0rc0, 2.18.0rc1, 2.18.0rc2, 2.18.0, 2.18.1, 2.19.0rc0, 2.19.0, 2.19.1, 2.20.0rc0, 2.20.0)
ERROR: No matching distribution found for tensorflow==2.15.1


In [2]:
import os, numpy as np, pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

from sklearn.utils import class_weight
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [3]:
!git clone https://github.com/SamarAlamri/Ratq_Model.git
%cd Ratq_Model
!ls


Cloning into 'Ratq_Model'...
remote: Enumerating objects: 6660, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 6660 (delta 1), reused 0 (delta 0), pack-reused 6657 (from 4)
Receiving objects: 100% (6660/6660), 501.55 MiB | 16.44 MiB/s, done.
Resolving deltas: 100% (27/27), done.
Updating files: 100% (11352/11352), done.
/content/Ratq_Model
erythema_efficientnet_final.h5	 Ratq_Model_final.ipynb
erythema_efficientnet_robust.h5  Ratq_Model_MobileNetV2.ipynb
erythema_mobilenet_robust.h5	 requirements.txt
erythema_model_final.h5		 SurgWound_Augmented
erythema_model_finetuned.h5	 SurgWound_Augmented_Exudate_3Class
exudate_model_final.h5		 SurgWound_Cleaned
Ratq_Model_EfficientNetB0.ipynb


In [15]:
DATA_ROOT = "SurgWound_Cleaned"

TRAIN_CSV = os.path.join(DATA_ROOT, "train_data.csv")
VAL_CSV   = os.path.join(DATA_ROOT, "validation_data.csv")
TEST_CSV  = os.path.join(DATA_ROOT, "test_data.csv")

assert os.path.exists(TRAIN_CSV), "Missing train_data.csv"
assert os.path.exists(VAL_CSV), "Missing validation_data.csv"
assert os.path.exists(TEST_CSV), "Missing test_data.csv"

print("✅ DATA_ROOT:", DATA_ROOT)
print("✅ CSVs found")


✅ DATA_ROOT: SurgWound_Cleaned
✅ CSVs found


In [16]:
def load_erythema(csv_path):
    df = pd.read_csv(csv_path)
    df = df[df["image_id"].str.contains("erythema", case=False, na=False)]
    df = df[df["answer"].str.lower() != "uncertain"]
    df["answer"] = df["answer"].astype(str).str.strip()
    return df.reset_index(drop=True)

train_df = load_erythema(TRAIN_CSV)
val_df   = load_erythema(VAL_CSV)
test_df  = load_erythema(TEST_CSV)

print("Train:\n", train_df["answer"].value_counts())
print("Val:\n", val_df["answer"].value_counts())
print("Test:\n", test_df["answer"].value_counts())


Train:
 answer
Non-existent    334
Existent        129
Name: count, dtype: int64
Val:
 answer
Non-existent    44
Existent        22
Name: count, dtype: int64
Test:
 answer
Non-existent    91
Existent        40
Name: count, dtype: int64


In [17]:
def fix_paths(df):
    df = df.copy()
    df["filename"] = df["filename"].astype(str)
    df["filename"] = df["filename"].str.replace("\\\\", "/", regex=False)
    df["filename"] = df["filename"].str.replace("\\", "/", regex=False)
    return df

train_df = fix_paths(train_df)
val_df   = fix_paths(val_df)
test_df  = fix_paths(test_df)

# sanity check
p = os.path.join(DATA_ROOT, train_df["filename"].iloc[0])
print("Example fixed filename:", train_df["filename"].iloc[0])
print("Exists?", os.path.exists(p))
assert os.path.exists(p), "❌ Still cannot find images. Paths or folder structure mismatch."
print("✅ Paths fixed correctly")


Example fixed filename: train/images/0.jpg_erythema.jpg
Exists? True
✅ Paths fixed correctly


In [18]:
IMG_SIZE = (300, 300)
BATCH_SIZE = 16

train_gen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=15,
    width_shift_range=0.08,
    height_shift_range=0.08,
    zoom_range=0.15,
    brightness_range=[0.90, 1.10],
    horizontal_flip=True
)

val_gen = ImageDataGenerator(rescale=1./255)

train_loader = train_gen.flow_from_dataframe(
    train_df, DATA_ROOT,
    x_col="filename", y_col="answer",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=True
)

val_loader = val_gen.flow_from_dataframe(
    val_df, DATA_ROOT,
    x_col="filename", y_col="answer",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=False
)

test_loader = val_gen.flow_from_dataframe(
    test_df, DATA_ROOT,
    x_col="filename", y_col="answer",
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode="sparse",
    shuffle=False
)

print("✅ Train batches:", len(train_loader))
print("✅ Class mapping:", train_loader.class_indices)
assert len(train_loader) > 0, "❌ Train loader is empty."
assert train_loader.class_indices, "❌ No class indices."


Found 463 validated image filenames belonging to 2 classes.
Found 66 validated image filenames belonging to 2 classes.
Found 131 validated image filenames belonging to 2 classes.
✅ Train batches: 29
✅ Class mapping: {'Existent': 0, 'Non-existent': 1}


In [19]:
y_train = np.array(train_loader.classes, dtype=int)
classes = np.array(list(train_loader.class_indices.values()), dtype=int)

weights = class_weight.compute_class_weight(
    class_weight="balanced",
    classes=classes,
    y=y_train
)

CLASS_WEIGHTS = {int(c): float(w) for c, w in zip(classes, weights)}

print("Class indices:", train_loader.class_indices)
print("Class weights:", CLASS_WEIGHTS)


Class indices: {'Existent': 0, 'Non-existent': 1}
Class weights: {0: 1.7945736434108528, 1: 0.6931137724550899}


In [20]:
def build_head(x, num_classes=2):
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(256, activation="relu")(x)
    x = layers.Dropout(0.5)(x)
    return layers.Dense(num_classes, activation="softmax")(x)

def compile_model(model, lr=1e-4):
    model.compile(
        optimizer=Adam(lr),
        loss="sparse_categorical_crossentropy",
        metrics=["accuracy"]
    )
    return model

def fit_model(model, epochs=8, lr=1e-4):
    compile_model(model, lr=lr)
    callbacks = [
        EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True),
        ReduceLROnPlateau(monitor="val_loss", patience=3, factor=0.5, min_lr=1e-6)
    ]
    model.fit(
        train_loader,
        validation_data=val_loader,
        epochs=epochs,
        class_weight=CLASS_WEIGHTS,
        callbacks=callbacks,
        verbose=1
    )

def eval_argmax(model):
    probs = model.predict(test_loader, verbose=0)
    y_pred = np.argmax(probs, axis=1)
    y_true = test_loader.classes
    return (accuracy_score(y_true, y_pred),
            f1_score(y_true, y_pred, average="weighted", zero_division=0))

def best_threshold_on_val(model):
    probs = model.predict(val_loader, verbose=0)
    y_val = val_loader.classes

    ex_idx = train_loader.class_indices["Existent"]
    p_ex = probs[:, ex_idx]

    best_t, best_f1 = 0.5, -1
    for t in np.arange(0.05, 0.96, 0.01):
        y_pred_t = np.where(p_ex >= t, ex_idx, 1 - ex_idx)
        f1w_t = f1_score(y_val, y_pred_t, average="weighted", zero_division=0)
        if f1w_t > best_f1:
            best_f1 = f1w_t
            best_t = t
    return best_t, best_f1

def eval_threshold(model, t):
    probs = model.predict(test_loader, verbose=0)
    y_true = test_loader.classes

    ex_idx = train_loader.class_indices["Existent"]
    p_ex = probs[:, ex_idx]
    y_pred = np.where(p_ex >= t, ex_idx, 1 - ex_idx)

    return (accuracy_score(y_true, y_pred),
            f1_score(y_true, y_pred, average="weighted", zero_division=0))


In [21]:
from tensorflow.keras.applications import (
    EfficientNetV2S,
    ConvNeXtTiny,
    DenseNet201,
    ResNet101V2,
    InceptionResNetV2
)

def make_model(backbone_name):
    inputs = layers.Input(shape=(IMG_SIZE[0], IMG_SIZE[1], 3))

    if backbone_name == "EfficientNetV2S":
        base = EfficientNetV2S(include_top=False, weights="imagenet", input_tensor=inputs)
        ft_last = 80
    elif backbone_name == "ConvNeXtTiny":
        base = ConvNeXtTiny(include_top=False, weights="imagenet", input_tensor=inputs)
        ft_last = 60
    elif backbone_name == "DenseNet201":
        base = DenseNet201(include_top=False, weights="imagenet", input_tensor=inputs)
        ft_last = 80
    elif backbone_name == "ResNet101V2":
        base = ResNet101V2(include_top=False, weights="imagenet", input_tensor=inputs)
        ft_last = 60
    elif backbone_name == "InceptionResNetV2":
        base = InceptionResNetV2(include_top=False, weights="imagenet", input_tensor=inputs)
        ft_last = 60
    else:
        raise ValueError("Unknown backbone")

    # stage 1 freeze
    base.trainable = False
    outputs = build_head(base.output, 2)
    model = models.Model(inputs, outputs, name=f"Erythema_{backbone_name}")

    # stage 2 unfreeze last N layers
    fine_tune_at = max(0, len(base.layers) - ft_last)
    return model, base, fine_tune_at


In [22]:
BACKBONES = ["EfficientNetV2S", "ConvNeXtTiny", "DenseNet201", "ResNet101V2", "InceptionResNetV2"]
results = []

for name in BACKBONES:
    print("\n" + "="*70)
    print("🚀 Training:", name)
    print("="*70)

    model, base, fine_tune_at = make_model(name)

    # Stage 1
    print("🧩 Stage 1: head warmup")
    fit_model(model, epochs=8, lr=1e-4)

    # Stage 2
    print("🔧 Stage 2: fine-tune")
    base.trainable = True
    for layer in base.layers[:fine_tune_at]:
        layer.trainable = False
    fit_model(model, epochs=15, lr=1e-5)

    acc_arg, f1_arg = eval_argmax(model)
    best_t, best_val_f1 = best_threshold_on_val(model)
    acc_thr, f1_thr = eval_threshold(model, best_t)

    results.append({
        "Model": name,
        "Argmax_ACC": acc_arg,
        "Argmax_F1W": f1_arg,
        "Best_t(VAL)": best_t,
        "Test_ACC@t": acc_thr,
        "Test_F1W@t": f1_thr
    })

    print(f"✅ {name} | Argmax ACC={acc_arg:.4f} F1W={f1_arg:.4f}")
    print(f"✅ {name} | Best t={best_t:.2f} -> TEST ACC={acc_thr:.4f} F1W={f1_thr:.4f}")



🚀 Training: EfficientNetV2S
82420632/82420632 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step
🧩 Stage 1: head warmup


/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 130s 3s/step - accuracy: 0.4237 - loss: 1.1953 - val_accuracy: 0.6212 - val_loss: 0.6816 - learning_rate: 1.0000e-04
Epoch 2/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 381ms/step - accuracy: 0.5259 - loss: 0.9782 - val_accuracy: 0.5152 - val_loss: 0.6922 - learning_rate: 1.0000e-04
Epoch 3/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 390ms/step - accuracy: 0.5193 - loss: 0.8467 - val_accuracy: 0.6212 - val_loss: 0.6919 - learning_rate: 1.0000e-04
Epoch 4/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 385ms/step - accuracy: 0.5050 - loss: 1.0475 - val_accuracy: 0.5606 - val_loss: 0.6934 - learning_rate: 1.0000e-04
Epoch 5/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 386ms/step - accuracy: 0.5324 - loss: 0.8142 - val_accuracy: 0.5606 - val_loss: 0.6954 - learning_rate: 5.0000e-05
Epoch 6/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 386ms/step - accuracy: 0.5049 - loss: 0.8889 - val_accuracy: 0.5455 - val_loss: 0.6921 - learning_rate: 5.0000e-05
Epoch 7/8
29/29 ━━━━━━━━━━━━━━━━━━━━ 11s 380ms/step - accuracy: 0.

In [23]:
results_df = pd.DataFrame(results).sort_values("Test_F1W@t", ascending=False)
results_df


,Model,Argmax_ACC,Argmax_F1W,Best_t(VAL),Test_ACC@t,Test_F1W@t
1,ConvNeXtTiny,0.687023,0.565784,0.43,0.633588,0.641820
3,ResNet101V2,0.625954,0.629580,0.67,0.664122,0.630480
0,EfficientNetV2S,0.610687,0.602915,0.50,0.610687,0.602915
2,DenseNet201,0.587786,0.603901,0.53,0.572519,0.585311
4,InceptionResNetV2,0.564885,0.552082,0.53,0.572519,0.548021


In [24]:
results_df.to_csv("erythema_5models_results.csv", index=False)
print("✅ Saved results to erythema_5models_results.csv")


✅ Saved results to erythema_5models_results.csv
